# Hubbard Model 

In [1]:
using ITensors
using ITensorMPS

In [17]:
sites = siteinds("Electron",5)

5-element Vector{Index{Int64}}:
 (dim=4|id=434|"Electron,Site,n=1")
 (dim=4|id=574|"Electron,Site,n=2")
 (dim=4|id=827|"Electron,Site,n=3")
 (dim=4|id=716|"Electron,Site,n=4")
 (dim=4|id=298|"Electron,Site,n=5")

In [18]:
random_mps(sites; linkdims=20)

MPS
[1] ((dim=4|id=434|"Electron,Site,n=1"), (dim=20|id=267|"Link,l=1"))
[2] ((dim=20|id=267|"Link,l=1"), (dim=4|id=574|"Electron,Site,n=2"), (dim=20|id=496|"Link,l=2"))
[3] ((dim=20|id=496|"Link,l=2"), (dim=4|id=827|"Electron,Site,n=3"), (dim=16|id=128|"Link,l=3"))
[4] ((dim=16|id=128|"Link,l=3"), (dim=4|id=716|"Electron,Site,n=4"), (dim=4|id=870|"Link,l=4"))
[5] ((dim=4|id=870|"Link,l=4"), (dim=4|id=298|"Electron,Site,n=5"))


In [19]:
Cup4 = op("Cup",sites[4])

ITensor ord=2 (dim=4|id=716|"Electron,Site,n=4")' (dim=4|id=716|"Electron,Site,n=4")
NDTensors.Dense{Float64, Vector{Float64}}

In [21]:
apply(Cup4, random_mps(sites; linkdims=20))

MPS
[1] ((dim=4|id=434|"Electron,Site,n=1"), (dim=4|id=44|"Link,l=1"))
[2] ((dim=4|id=574|"Electron,Site,n=2"), (dim=4|id=44|"Link,l=1"), (dim=16|id=232|"Link,l=2"))
[3] ((dim=4|id=827|"Electron,Site,n=3"), (dim=16|id=232|"Link,l=2"), (dim=16|id=309|"Link,l=3"))
[4] ((dim=4|id=716|"Electron,Site,n=4"), (dim=4|id=335|"Link,l=4"), (dim=16|id=309|"Link,l=3"))
[5] ((dim=4|id=335|"Link,l=4"), (dim=4|id=298|"Electron,Site,n=5"))
